In [1]:
%load_ext autoreload
%aimport data_creater
%aimport model
%autoreload 1

from data_creater import *
from model import *
import re, copy

Using TensorFlow backend.


In [3]:
stocks = companies()
symbols = stocks['Symbol'].values.tolist()

In [13]:
import gc 
# del df
# del stocks, Sequential
gc.collect()

199

In [5]:
symbol = 'MMM'
other_symbols = [symbol_tem for symbol_tem in symbols if symbol_tem != symbol]
df = pd.read_csv('./data/{0}/all_normalized.csv'.format(symbol), index_col=[0], parse_dates=[0])
symbol_columns = [symbol_tem for symbol_tem in list(df.columns.values) if bool(re.match('normal.*', symbol_tem))]

box = []
for symbol_tem in other_symbols:
    box_tem = []
    for col in list(df.columns.values):
        if bool(re.match(symbol_tem + '_normal_.*', col)):
            box_tem.append(col)
    box.append(box_tem)

all_combination = []
all_combination.append(symbol_columns)
for box_tem in box:
    all_combination.append(symbol_columns + box_tem)
    
all_combination[0]

['normal_amplitude',
 'normal_volume',
 'normal_close',
 'normal_close_diff_day1',
 'normal_close_diff_day2',
 'normal_close_diff_day3',
 'normal_returns',
 'normal_mfi',
 'normal_weekday']

In [6]:
ticker = 'AAPL'
window_size = 3
seq_obj = MultiSequence(ticker,window_size,1,'g000',all_combination[0])


# sys.exit()
X_train,y_train,X_test,y_test = split_data(seq_obj)
print(seq_obj.X[2])
print(seq_obj.y[2])





[[-0.03352107 -0.17754371 -0.3861563   0.70570341  0.54774041  0.66382023
   0.64942526 -0.68993871  3.        ]
 [-0.26815705 -0.2792827  -0.40817555  0.09049326  0.47626233  0.44672878
   0.12399471 -0.64910956  4.        ]
 [-0.60335345 -0.33233866 -0.42354147  0.12243274 -0.06654407  0.41229764
   0.15052712 -0.64951155  5.        ]]
[1]


In [7]:
window_sizes = [3,4]
dropouts =  [0.25]
learn_rates = [0.001]
epochs = [5,10,30]
batch_size = 50

In [ ]:
result = model_selector('AAPL', window_sizes, learn_rates, dropouts, epochs, batch_size,target_length=1,target_theme='g001',verbose=1,column=all_combination[1])

print("\nResults : ")
print("-"*60)
print(result[0])

print(result[1])
ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)


*** Best Model Selection for AAPL ***

Window size: 3
------------------------------------------------------------
1    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 5 Training loss: 4.2359 Training acc: 0.6632 Testing loss: 4.7219 Testing acc: 0.3469 Score : 0.4556
2    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 10 Training loss: 0.6138 Training acc: 0.7254 Testing loss: 0.6264 Testing acc: 0.6939 Score : 0.7093
3    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 30 Training loss: 0.6637 Training acc: 0.7513 Testing loss: 0.9840 Testing acc: 0.6939 Score : 0.7214

Window size: 4
------------------------------------------------------------
4    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 5 Training loss: 0.6055 Training acc: 0.6771 Testing loss: 0.9504 Testing acc: 0.6939 Score : 0.6854
5    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 10 Training loss: 0.6894 Training acc: 0.7083 Testing loss: 2.1238 Testing acc: 0.6939 Score : 0.7010


In [30]:
result = model_selector('MMM', window_sizes, learn_rates, dropouts, epochs, batch_size,target_length=1,all_day=False,verbose=1,column=all_combination[0])

print("\nResults : ")
print("-"*60)
print(result[0])

print(result[1])
ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)



*** Best Model Selection for MMM ***

Window size: 1
------------------------------------------------------------
1    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 500 Training error: 0.0041 Testing error: 0.0065
2    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 700 Training error: 0.0040 Testing error: 0.0066
3    > Learn rate: 0.0010 Dropout: 0.40 Epoch: 500 Training error: 0.0042 Testing error: 0.0067
4    > Learn rate: 0.0010 Dropout: 0.40 Epoch: 700 Training error: 0.0040 Testing error: 0.0066

Window size: 2
------------------------------------------------------------
5    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 500 Training error: 0.0039 Testing error: 0.0067
6    > Learn rate: 0.0010 Dropout: 0.25 Epoch: 700 Training error: 0.0038 Testing error: 0.0074
7    > Learn rate: 0.0010 Dropout: 0.40 Epoch: 500 Training error: 0.0041 Testing error: 0.0071
8    > Learn rate: 0.0010 Dropout: 0.40 Epoch: 700 Training error: 0.0037 Testing error: 0.0075

Model selection summary for MMM with win

'Accuracy is lower than the past'

In [ ]:
window_sizes = [2, 5, 6, 7]
dropouts =  [0.25, 0.4]
learn_rates = [0.001]
epochs = [500,1000,1500]
batch_size = 30

for i in range(len(all_combination)-1):
    print(all_combination[i])
    print("{0} {1} {2}".format("-"*30, i + 1, "-"*30))
    result = model_selector(symbols[2], window_sizes, learn_rates, dropouts, epochs, batch_size,target_length=1,all_day=False,verbose=1,column=all_combination[i])

    print("\nResults : ")
    print("-"*60)
    print(result[0])

    print(result[1])
    ModelLoader.save(result[1]['ticker'],result[0],result[1],force_overwrite=False)

In [50]:
ticker = 'AAPL'
window_size = 1
seq_obj = MultiSequence(ticker,window_size,1,'g000',all_combination[0])
X = seq_obj.data
Y = pd.Series(seq_obj.ans)
X['close'] = Y
X

,normal_open_,normal_high,normal_low,normal_volume,normal_close,normal_returns,normal_mfi,close
0,-0.485243,-0.500641,-0.524723,-0.202762,-0.476723,0.368477,-0.727863,1
1,-0.531683,-0.503716,-0.514295,-0.288351,-0.482854,0.187972,-0.714509,0
2,-0.442885,-0.392330,-0.446182,-0.177544,-0.374746,0.649425,-0.689939,1
3,-0.342009,-0.366533,-0.399596,-0.279283,-0.397174,0.123995,-0.649110,0
4,-0.382325,-0.408730,-0.410865,-0.332339,-0.412826,0.150527,-0.649512,0
...,...,...,...,...,...,...,...,...
238,0.918687,0.909456,0.906660,-0.819771,0.906154,0.218105,0.630991,1
239,0.952199,0.958315,0.952741,-0.706738,0.954633,0.345839,0.634588,1
240,0.951348,0.959853,0.931551,-0.854600,0.966261,0.244761,0.464988,1
241,0.944714,0.994533,0.955600,-0.784475,1.000000,0.304359,0.469215,1


In [57]:
X = pd.DataFrame(seq_obj.X)
Y = seq_obj.y

model = Sequential()
model.add(Dense(12, input_dim=X.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# 2. 编译模型
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 3. 训练模型
history = model.fit(X, Y, nb_epoch=100, batch_size=10)
# 4. 评估模型
loss, accuracy = model.evaluate(X, Y)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
# 5. 数据预测
probabilities = model.predict(X)
predictions = [float(round(x)) for x in probabilities]
accuracy = numpy.mean(predictions == Y)
print("Prediction Accuracy: %.2f%%" % (accuracy*100))

/Users/bacon_huang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/100
242/242 [==============================] - 0s 506us/step - loss: 0.7705 - accuracy: 0.4298
Epoch 2/100
242/242 [==============================] - 0s 104us/step - loss: 0.7471 - accuracy: 0.4256
Epoch 3/100
242/242 [==============================] - 0s 105us/step - loss: 0.7292 - accuracy: 0.4050
Epoch 4/100
242/242 [==============================] - 0s 103us/step - loss: 0.7186 - accuracy: 0.4174
Epoch 5/100
242/242 [==============================] - 0s 94us/step - loss: 0.7119 - accuracy: 0.4545
Epoch 6/100
242/242 [==============================] - 0s 78us/step - loss: 0.7064 - accuracy: 0.4463
Epoch 7/100
242/242 [==============================] - 0s 80us/step - loss: 0.7022 - accuracy: 0.4752
Epoch 8/100
242/242 [==============================] - 0s 74us/step - loss: 0.6988 - accuracy: 0.4835
Epoch 9/100
242/242 [==============================] - 0s 76us/step - loss: 0.6958 - accuracy: 0.4959
Epoch 10/100
242/242 [==============================] - 0s 78us/step - loss: 0

TypeError: type numpy.ndarray doesn't define __round__ method

In [8]:
from keras import backend as K
import tensorflow as tf

for ticker in symbols[1:]:
    #release memory
    K.clear_session()
    tf.reset_default_graph()
    
    result = model_selector(ticker, window_sizes, learn_rates, dropouts, epochs, batch_size,verbose=2)
    
    #save trained model
    ModelLoader.save(result[1]['ticker'],result[0],result[1])
    print(" ==> Saved trained model for {}".format(result[1]['ticker']))


Model selection summary for AXP with window size of 10:
------------------------------------------------------------
 ==> Learn rate: 0.0100 Dropout: 0.40 Epoch: 100 Training error: 0.0035 Testing error: 0.0053
 ==> Saved trained model for AXP

Model selection summary for AAPL with window size of 7:
------------------------------------------------------------
 ==> Learn rate: 0.0100 Dropout: 0.40 Epoch: 100 Training error: 0.0031 Testing error: 0.0109
 ==> Saved trained model for AAPL

Model selection summary for BA with window size of 7:
------------------------------------------------------------
 ==> Learn rate: 0.0100 Dropout: 0.40 Epoch: 100 Training error: 0.0059 Testing error: 0.0031
 ==> Saved trained model for BA

Model selection summary for CAT with window size of 7:
------------------------------------------------------------
 ==> Learn rate: 0.0010 Dropout: 0.25 Epoch: 200 Training error: 0.0030 Testing error: 0.0055
 ==> Saved trained model for CAT

Model selection summar